# Embedding
It is a way to turn words into meaningful numbers. <br /> It stores how a word is used. It can capture things like which words appear near it, what it can replace, and what usually comes before or after it.

cat is close to dog pet and animal. <br />
food is close to drink, cook. <br /> <br />
These embeddings are learned automatically. <br /> <br />

`block_size` : How many tokens the model procress in one forward pass. <br />
In the example below we are taking 15 data in a chunk.


In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

In [13]:
import string


data = string.printable

def get_chunk(block_size, idx):
    x = data[idx:idx + block_size]
    y = data[idx + 1:idx + block_size + 1]
    return x, y

x, y = get_chunk(12, 5)

### Input and expected output
We have the above function cause we want to predict the next token in our character level model. So the code below gives the example of what we might expect as output `y` when our input is `x`. <br />
<b>Character based model.</b>

In [47]:
from src.tokenizer import Tokenizer
import torch

tokenizer = Tokenizer(data)

phrase = torch.tensor(tokenizer.encode("full-stack application"))

block_size = 10

for i in range(0, block_size):
    input_arr = phrase[:i + 1]
    print(f"when the input is {input_arr} output is {phrase[i + 1]}")



when the input is tensor([15]) output is 30
when the input is tensor([15, 30]) output is 21
when the input is tensor([15, 30, 21]) output is 21
when the input is tensor([15, 30, 21, 21]) output is 74
when the input is tensor([15, 30, 21, 21, 74]) output is 28
when the input is tensor([15, 30, 21, 21, 74, 28]) output is 29
when the input is tensor([15, 30, 21, 21, 74, 28, 29]) output is 10
when the input is tensor([15, 30, 21, 21, 74, 28, 29, 10]) output is 12
when the input is tensor([15, 30, 21, 21, 74, 28, 29, 10, 12]) output is 20
when the input is tensor([15, 30, 21, 21, 74, 28, 29, 10, 12, 20]) output is 94


In [15]:
import torch.nn as nn


class Transformer(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.block_size = 25
        self.d_model = 128
        self.vocab_size = 75

        self.p_e = nn.Embedding(self.block_size, self.d_model)
        self.t_e = nn.Embedding(self.vocab_size, self.d_model)

transformer = Transformer()
print(transformer.p_e)
print(transformer.t_e)

Embedding(25, 128)
Embedding(75, 128)
